# A Voice Assistant for Your Knowledge Base

• Find the link to the “[JarvisBase](https://github.com/peterw/JarvisBase)” GitHub repository for this section at  [towardsai.net/book](http://towardsai.net/book).

This tutorial focuses on voice capabilities. In this project, we will create a voice assistant that integrates OpenAI’s Whisper to convert voice inputs into text. After the transcription is complete, voice responses will be generated using Eleven Labs, a company renowned for its high-quality text-to-speech API that adeptly captures emotion and tone. Using this API will ensure that the voice assistant can communicate with users in a clear and natural tone.

At the heart of this project is a question-answering system. When a question is asked, the system retrieves relevant documents from this database. These documents and the question are then processed by a large language model (LLM). The LLM utilizes this information to formulate an appropriate response.

The project includes the Streamlit service to create an interactive user interface (UI), enhancing user interaction with the assistant. This basic frontend allows users to ask questions using either natural language or voice and generates responses in both text and audio formats.

![image](./images/jarvis-base.jpg)

Start by installing the necessary libraries for this project. While it’s best to use the most recent versions of these packages for the best results, the provided code was used with specific versions. They can be installed using the pip packages manager. A link to this requirement file is accessible at [towardsai.net/book](http://towardsai.net/book).

Set the API keys and tokens. They need to be set in the environment variable as described below.

In [ ]:
import os
from langchain_custom_utils.helper import get_openai_api_key, get_activeloop_api_key, get_eleven_api_key
OPENAI_API_KEY = get_openai_api_key()
ACTIVELOOP_API_KEY = get_activeloop_api_key()
ELEVEN_API_KEY = get_eleven_api_key()